<a href="https://colab.research.google.com/github/Ignas12345/Magistro_projektas/blob/main/knygutes/keliu_palyginimas/keliu_palyginimas_ir_filtravimas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
def delete_targets_not_in_list(current_targets, list_to_keep):
    current_targets =  current_targets.split(',')
    #sekancios dvi eilutes tam, kad panaikintu tarpa, nes kazkodel tame keliu excelio faile, antras kelyje dalyvaujantis genas turi tarpa pries pavadinima:
    if len(current_targets) > 1:
      current_targets[1] = current_targets[1].replace(' ', '')
    targets_to_keep = [target for target in current_targets if target in list_to_keep]
    #num_deleted = len(current_targets) - len(targets_to_keep)
    #print('Deleted ' + str(num_deleted) + ' targets')
    return ','.join(targets_to_keep)

def delete_targets_in_list(current_targets, list_to_delete):
    current_targets =  current_targets.split(',')
    targets_to_keep = [target for target in current_targets if target not in list_to_delete]
    #num_deleted = len(current_targets) - len(targets_to_keep)
    #print('Deleted ' + str(num_deleted) + ' targets')
    return ','.join(targets_to_keep)

'''
def get_annotations(annotations_list):
  annotations_dict = {}
  for item in annotations_list:
      if "." in item:  # Skip empty strings
          key = item.split('.')[-1]  # Extract the last part (e.g., "TRIOBP")
          annotations_dict[key] = item  # Map the key to the full annotation
  return annotations_dict
'''

def annotate_targets(targets, annotations_dict):
    targets_list = targets.split(',')  # Split targets into a list
    annotated_list = [annotations_dict.get(target, target) for target in targets_list]  # Annotate
    return ','.join(annotated_list)

**Čia palyginami keliai iš miRPath duombazės.** Man regis, kad atrasti kelius, kuriuos reguliuoja tam tikra miRNR, jie naudojo Enrichement analysis naudodami kelius iš šitų duombazių: KEGG, Reactome, WikiPathways, Gene Ontology (https://doi.org/10.1093/nar/gkz1022). Tiesiog pasižiūrimi bendri keliai ir padaroma nefiltruota lentelė.

In [ ]:
have_comparison = 0

#kiek geriausiai diferencijuojanciu genu naudojam, naudojau 4000
n = 18490
str_1 = '371a-3p'
str_2= '20a-5p'

if have_comparison == 1:
  filtered_comparison_df = pd.read_csv('/content/initial_comparison_'+str_1+'_and_'+str_2+'.csv')
else:
  df_1 = pd.read_csv('/content/' + str_1 +'_paths.csv')
  df_2 = pd.read_csv('/content/' + str_2 +'_paths.csv')
  #str_2 += "_"
  intersection = pd.merge(df_1, df_2, on='Pathway')
  sorted_intersection = intersection.sort_values(by=['P-value_x', 'P-value_y'], ascending=[True, True])
  filtered_intersection = sorted_intersection.drop_duplicates(subset=['Pathway'], keep='first').copy()
  filtered_intersection.loc[:,'number_of_genes_regulated_in_pathway'] = filtered_intersection['Hits_x'] + filtered_intersection['Hits_y']
  #filtered_intersection.loc[:,'Score'] = 0
  #filtered_intersection = filtered_intersection.sort_values(by='total_number_of_genes_regulated_in_pathway')
  pathway_comparison_df = filtered_intersection.drop(columns=['Database_y'], inplace=False)
  pathway_comparison_df.rename(columns={'Database_x': 'Database'}, inplace=True)
  pathway_comparison_df.rename(columns={'Evidence_x': 'Evidence_' + str_1, 'Hits_x': 'Hits_' + str_1, 'Expected hits_x': 'Expected hits_' + str_1, 'P-value_x': 'P-value_' + str_1, 'Targets_x': 'Targets_' + str_1}, inplace=True)
  pathway_comparison_df.rename(columns={'Evidence_y': 'Evidence_' + str_2, 'Hits_y': 'Hits_' + str_2, 'Expected hits_y': 'Expected hits_' + str_2, 'P-value_y': 'P-value_' + str_2, 'Targets_y': 'Targets_' + str_2}, inplace=True)
  #str_2 = "371a-3p"
  pathway_comparison_df.reset_index(drop=True, inplace=True)

Čia iš kitos knygutės atsirenkami atfiltruoti taikiniai, t.y., taikiniai, kurie buvo paiimti iš kitų duombazių ir palikti n geriausiai (turinčių mažiausią padj iš DE atliktos kitoje knygutėje) diferencijuojančiais genais.

In [ ]:
targets_1_df = pd.read_csv('/content/targets_'+str_1+'.csv')
targets_2_df = pd.read_csv('/content/targets_'+str_2+'.csv')
genes_df = pd.read_csv('/content/ordered_genes.csv')
best_differentiating_genes_list = genes_df['gene_name'].tolist()[:n]

In [ ]:
#Pasiliksime tik taikinius, kurie patenka tarp n geriausiai musu duomenis diferencijuojanciu genu
differentiating_targets_1_df = targets_1_df[targets_1_df['gene_name'].isin(best_differentiating_genes_list)].copy()
differentiating_targets_2_df = targets_2_df[targets_2_df['gene_name'].isin(best_differentiating_genes_list)].copy()
differentiating_targets_1_set = set(differentiating_targets_1_df['gene_name'])
differentiating_targets_2_set = set(differentiating_targets_2_df['gene_name'])
len(differentiating_targets_1_set), len(differentiating_targets_2_set)

(1272, 1898)

In [ ]:
#cia sukuriam sarasa geriausiai koreliuojanciu genu

threshold = -0.2
correlated_genes_1_df = pd.read_csv('/content/spearman_correlations_'+str_1+'.csv')
correlated_genes_2_df = pd.read_csv('/content/spearman_correlations_'+str_2+'.csv')
correlated_genes_1_set = set(correlated_genes_1_df[correlated_genes_1_df['spearman_correlation'] < threshold]['gene_name'])
correlated_genes_2_set = set(correlated_genes_2_df[correlated_genes_2_df['spearman_correlation'] < threshold]['gene_name'])
len(correlated_genes_1_set), len(correlated_genes_2_set)

(6401, 4189)

In [ ]:
#galiausiai pasiliekam tik taikinius, kurie ir yra taikiniai duombazėse ir gerai diferencijuoja ir koreliuoja
target_list_1 = list(differentiating_targets_1_set.intersection(correlated_genes_1_set))
target_list_2 = list(differentiating_targets_2_set.intersection(correlated_genes_2_set))

len(target_list_1), len(target_list_2)


(509, 416)

Dabar paliekam tik tuos taikinius, kurie yra tame mūsų sąrašę

In [ ]:
filtered_comparison_df = pathway_comparison_df.copy()
filtered_comparison_df['Targets_'+str_1] = filtered_comparison_df['Targets_'+str_1].apply(lambda x: delete_targets_not_in_list(x, target_list_1))
filtered_comparison_df['Targets_'+str_2] = filtered_comparison_df['Targets_'+str_2].apply(lambda x: delete_targets_not_in_list(x, target_list_2))

filtered_comparison_df = filtered_comparison_df[filtered_comparison_df['Targets_'+str_1] != ''].copy()
filtered_comparison_df = filtered_comparison_df[filtered_comparison_df['Targets_'+str_2] != ''].copy()
filtered_comparison_df['Num_targets_after_filtering'] = filtered_comparison_df.apply(lambda row: len(row['Targets_'+str_1].split(',')) + len(row['Targets_'+str_2].split(',')), axis=1)
filtered_comparison_df.sort_values(by='Num_targets_after_filtering', inplace=True)
filtered_comparison_df.reset_index(drop=True, inplace=True)

write_to_file = 0
if write_to_file == 1:
  filtered_comparison_df.to_csv('initial_comparison_'+str_1+'_and_'+str_2+'.csv')
#filtered_comparison_df


Pridedam prie taikiniu ju koreliacija su miRNR ir DE geba.

In [ ]:
#write a function that calculates in how many rows a gene appears:
def get_num_rows_with_genes(df, str_1, str_2):
    genes_counts_dict = {}
    genes_list = list((set(df['Targets_'+str_1].str.split(',').explode())).union(set(df['Targets_'+str_2].str.split(',').explode())))
    for gene in genes_list:
        genes_counts_dict[gene] = df[df['Targets_'+str_1].str.contains(gene) | df['Targets_'+str_2].str.contains(gene)].shape[0]
    return genes_counts_dict

In [ ]:
# Precompute correlation and padj mappings for faster lookups
correlation_dict_1 = correlated_genes_1_df.set_index('gene_name')['spearman_correlation'].to_dict()
spearman_pvalue_dict_1 = correlated_genes_1_df.set_index('gene_name')['p-value'].to_dict()
TLink_dict_1 = targets_1_df.set_index('gene_name')['in_TargetLink'].to_dict()
correlation_dict_2 = correlated_genes_2_df.set_index('gene_name')['spearman_correlation'].to_dict()
spearman_pvalue_dict_2 = correlated_genes_2_df.set_index('gene_name')['p-value'].to_dict()
TLink_dict_2 = targets_2_df.set_index('gene_name')['in_TargetLink'].to_dict()
padj_dict = genes_df.set_index('gene_name')['padj'].to_dict()
genes_counts_dict = get_num_rows_with_genes(filtered_comparison_df, str_1, str_2)

def process_targets(targets, correlation_dict, spearman_pvalue_dict,padj_dict, TLink_dict, genes_counts_dict):
    if pd.isna(targets):  # Handle NaN targets gracefully
        return ""
    result = []
    for target in targets.split(','):
        target_corr = correlation_dict.get(target, "NA")  # Default to "NA" if target not found
        spearman_pvalue = spearman_pvalue_dict.get(target, "NA")
        target_padj = padj_dict.get(target, "NA")
        has_TLink = TLink_dict.get(target, "NA")
        num_repeats = genes_counts_dict.get(target, 0)
        result.append(f"{target}{{{float(target_corr):.2f};{float(spearman_pvalue):.0e};{float(target_padj):.0e}; {has_TLink}; {num_repeats}}}")
    return ','.join(result)

# Apply the function to compute results for both columns
filtered_comparison_df[f'Spearman_corrs_and_pvalues_and_DE_pvalues_and_TLink_and_num_paths_{str_1}'] = filtered_comparison_df[f'Targets_{str_1}'].apply(
    lambda x: process_targets(x, correlation_dict_1, spearman_pvalue_dict_1, padj_dict, TLink_dict_1, genes_counts_dict)
)

filtered_comparison_df[f'Spearman_corrs_and_pvalues_and_DE_pvalues_and_TLink_and_num_paths_{str_2}'] = filtered_comparison_df[f'Targets_{str_2}'].apply(
    lambda x: process_targets(x, correlation_dict_2, spearman_pvalue_dict_2, padj_dict, TLink_dict_2, genes_counts_dict)
)

filtered_comparison_df['common_targets'] = filtered_comparison_df.apply(
    lambda row: set(str(row[f'Targets_{str_1}']).split(',')).intersection(
        set(str(row[f'Targets_{str_2}']).split(','))), axis=1)

write_to_file = 1
if write_to_file == 1:
  filtered_comparison_df.to_csv('initial_comparison_'+str_1+'_and_'+str_2+'.csv')
#filtered_comparison_df

Čia vykdomas automatinis sužymėjimas.

In [ ]:
def create_annotations_dict(target_list, correlation_dict, spearman_pvalue_dict, differentiation_dict, weak_differentiation_threshold, strong_differetiation_threshold, weak_correlation_threshold, strong_correlation_threshold, weak_correlation_pvalue_threshold, strong_correlation_pvalue_threshold):
  annotations_dict = {}
  for gene in target_list:
    annotations_dict[str(gene)] = ""
    if correlation_dict[str(gene)] > weak_correlation_threshold and spearman_pvalue_dict[str(gene)] > weak_correlation_pvalue_threshold:
      annotations_dict[str(gene)] += "weak.corr."
    elif correlation_dict[str(gene)] < strong_correlation_threshold and spearman_pvalue_dict[str(gene)] < strong_correlation_pvalue_threshold:
      annotations_dict[str(gene)] += "strong.corr."
    if differentiation_dict[str(gene)] > weak_differentiation_threshold:
      annotations_dict[str(gene)] += "weak.diff."
    elif differentiation_dict[str(gene)] < strong_differetiation_threshold:
      annotations_dict[str(gene)] += "strong.diff."
    annotations_dict[str(gene)] += str(gene)

  return annotations_dict

In [ ]:
weak_differentiation_threshold = 0.1
strong_differetiation_threshold = 0.001
weak_correlation_threshold = -0.3
strong_correlation_threshold = -0.5
weak_correlation_pvalue_threshold = 0.001
strong_correlation_pvalue_threshold = 1e-9

In [ ]:
annotations_dict_1 = create_annotations_dict(target_list_1, correlation_dict_1, spearman_pvalue_dict_1, padj_dict, weak_differentiation_threshold = weak_differentiation_threshold, strong_differetiation_threshold = strong_differetiation_threshold, weak_correlation_threshold = weak_correlation_threshold, strong_correlation_threshold = strong_correlation_threshold, weak_correlation_pvalue_threshold = weak_correlation_pvalue_threshold, strong_correlation_pvalue_threshold = strong_correlation_pvalue_threshold)
annotations_dict_2 = create_annotations_dict(target_list_2, correlation_dict_2, spearman_pvalue_dict_2, padj_dict,weak_differentiation_threshold = weak_differentiation_threshold, strong_differetiation_threshold = strong_differetiation_threshold, weak_correlation_threshold = weak_correlation_threshold, strong_correlation_threshold = strong_correlation_threshold, weak_correlation_pvalue_threshold = weak_correlation_pvalue_threshold, strong_correlation_pvalue_threshold = strong_correlation_pvalue_threshold)

In [ ]:
annotated_df = filtered_comparison_df.copy()
annotated_df['Targets_'+str_1] = annotated_df['Targets_'+str_1].apply(annotate_targets, args=(annotations_dict_1,))
annotated_df['Targets_'+str_2] = annotated_df['Targets_'+str_2].apply(annotate_targets, args=(annotations_dict_2,))
#annotated_df['Targets_'+str_1] = annotated_df['Targets_'+str_1].apply(annotate_targets, args=(non_differentiating_dict,))
#annotated_df['Targets_'+str_2] = annotated_df['Targets_'+str_2].apply(annotate_targets, args=(non_differentiating_dict,))

write_to_file = 1
if write_to_file == 1:
  annotated_df.to_csv('annotated_comparison_'+str_1+'_and_'+str_2+'.csv')
#annotated_df.head(n = 15)


Čia ištrinami iš lentelės "iš akies" nediferencijuojantys ar tiesiogiai nereguliuojami genai

In [ ]:
def remove_with_certain_annotations(targets):
    # Split the string into individual genes
    genes = targets.split(',')
    # Keep only genes that don't match the unwanted patterns
    filtered_genes = [gene for gene in genes if (not (gene.startswith('weak.corr.') or gene.startswith('weak.diff.')) or gene.startswith('weak.corr.strong.diff.'))]
    # Rejoin the list into a single string
    return ','.join(filtered_genes)

In [ ]:
viewed = 0
if viewed == 1:
  annotated_df = pd.read_csv('/content/manually_annotated_comparison_'+str_1+'_and_'+str_2+'.csv')
sifted_df = annotated_df.copy()
sifted_df['Targets_'+str_1] = sifted_df['Targets_'+str_1].apply(remove_with_certain_annotations)
sifted_df['Targets_'+str_2] = sifted_df['Targets_'+str_2].apply(remove_with_certain_annotations)
sifted_df = sifted_df[sifted_df['Targets_'+str_1] != ''].copy()
sifted_df = sifted_df[sifted_df['Targets_'+str_2] != ''].copy()

write_to_file = 1
if write_to_file == 1:
  sifted_df.to_csv('sifted_comparison_'+str_1+'_and_'+str_2+'.csv')
sifted_df

,Database,Pathway,Evidence_371a-3p,Hits_371a-3p,Expected hits_371a-3p,P-value_371a-3p,Targets_371a-3p,Evidence_20a-5p,Hits_20a-5p,Expected hits_20a-5p,P-value_20a-5p,Targets_20a-5p,number_of_genes_regulated_in_pathway,Num_targets_after_filtering,Spearman_corrs_and_pvalues_and_DE_pvalues_and_TLink_and_num_paths_371a-3p,Spearman_corrs_and_pvalues_and_DE_pvalues_and_TLink_and_num_paths_20a-5p,common_targets
0,Gene Ontology - Biological Process,tricuspid valve development,predicted (union),6,1.819,0.044000,BMPR2,experimental (strong),2,0.100,0.025000,strong.diff.TGFBR2,8,3,"BMPR1A{-0.23;6e-03;2e-01; 0; 71},BMPR2{-0.38;4...",TGFBR2{-0.44;5e-08;6e-04; 1; 67},{}
1,Gene Ontology - Cellular Component,transcriptional repressor complex,predicted (union),36,22.130,0.018000,strong.corr.ZBTB16,predicted (union),55,39.980,0.014000,JAZF1,91,4,ZBTB16{-0.61;3e-15;6e-03; 0; 35},"CCND1{-0.25;4e-03;4e-02; 1; 51},JAZF1{-0.33;9e...",{}
3,WikiPathways,Structural Pathway of Interleukin 1 (IL-1),predicted (union),24,13.340,0.020000,strong.corr.strong.diff.MAPK14,predicted (intersection),27,14.350,0.019000,"strong.corr.IRAK4,strong.diff.MAP3K3",51,5,"MAPK14{-0.51;2e-10;2e-04; 0; 81},MKNK1{-0.41;6...","IRAK4{-0.51;1e-10;3e-02; 1; 21},MAP3K3{-0.29;5...",{}
7,WikiPathways,ErbB Signaling Pathway,predicted (union),41,24.250,0.009000,strong.corr.weak.diff.CDKN1A,predicted (union),59,43.810,0.042000,"strong.diff.EGFR,strong.diff.PIK3R1",100,6,"AKT2{-0.26;2e-03;7e-02; 0; 61},CDKN1A{-0.55;3e...","CCND1{-0.25;4e-03;4e-02; 1; 51},CDKN1A{-0.42;2...",{CDKN1A}
8,KEGG,Transcriptional misregulation in cancer,experimental (any),3,0.355,0.021000,strong.corr.weak.diff.FLT1,experimental (strong),7,2.307,0.024000,"RUNX1,strong.diff.TGFBR2",10,6,"FLT1{-0.68;5e-20;2e-01; 1; 47},MYC{-0.24;4e-03...","CDKN1A{-0.42;2e-07;1e-01; 1; 54},MYC{-0.25;3e-...",{MYC}
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,Gene Ontology - Biological Process,regulation of signal transduction,predicted (union),944,855.900,0.007000,"strong.corr.strong.diff.AGO3,AKAP13,AKAP6,stro...",experimental (any),230,193.900,0.046000,"AAK1,weak.corr.strong.diff.AGO3,BCL2,BMP8B,str...",1174,156,"AGO3{-0.63;6e-17;2e-06; 0; 12},AKAP13{-0.33;6e...","AAK1{-0.32;2e-04;3e-03; 1; 21},AGO3{-0.27;1e-0...","{RUNX1, PTGIS, CNKSR3, JAK1, ARAP2, SQSTM1, ST..."
170,Gene Ontology - Biological Process,response to organic substance,predicted (union),899,824.000,0.028000,"strong.corr.strong.diff.ADD1,AKAP13,AKAP6,ATP6...",experimental (any),226,188.700,0.036000,"weak.corr.strong.diff.ADD1,BCL2,weak.corr.stro...",1125,160,"ADD1{-0.61;3e-15;6e-10; 1; 12},ADH5{-0.26;2e-0...","ADD1{-0.24;5e-03;6e-10; 1; 12},ATP6V0E1{-0.29;...","{KCNMB1, RUNX1, ADD1, CNKSR3, JAK1, SQSTM1, ST..."
171,Gene Ontology - Biological Process,anatomical structure morphogenesis,predicted (union),761,661.800,0.000752,"strong.corr.strong.diff.ADD1,AKAP13,strong.dif...",predicted (intersection),828,711.900,0.000020,"strong.corr.weak.diff.ACVRL1,weak.corr.strong....",1589,165,"ADD1{-0.61;3e-15;6e-10; 1; 12},AKAP13{-0.33;6e...","ACVRL1{-0.58;1e-13;1e-01; 0; 43},ADD1{-0.24;5e...","{ADD1, RUNX1, DOK6, FLT1, OSR1, BCL11B, JAK1, ..."
172,Gene Ontology - Biological Process,intracellular signal transduction,predicted (union),764,673.400,0.002000,"strong.corr.strong.diff.AGO3,AKAP13,AKAP6,stro...",predicted (intersection),830,724.200,0.000120,"ADRA1B,weak.corr.strong.diff.AGO3,ARHGAP31,wea...",1594,172,"AGO3{-0.63;6e-17;2e-06; 0; 12},AKAP13{-0.33;6e...","ADRA1B{-0.31;2e-04;2e-02; 0; 8},AGO3{-0.27;1e-...","{FLT1, CNKSR3, FAM13A, FIBIN, SRGAP3, ARAP2, N..."
